In [1]:
import glob
import os
import statistics
import numpy as np
import pandas as pd
from typing import Dict

In [3]:
# Read data
IH = pd.read_csv('data/FormantShiftIH.csv', sep=",", quotechar="'")

# Compute perturbation value (actual - feedback)/actual
IH["perturbation_value"] = (IH.Avg_F1-IH.S_Avg_F1)/IH.Avg_F1

# Remove unnecessary columns
clean = IH[["subject_id", "PROCESS_ORDER", "phasename", "num_phase", 
            "num_rep", "Avg_F1", "S_Avg_F1", "perturbation_value"]]

# Keep only start, ramp and stay phase rows
keep_phases = ["start", "ramp", "stay", "end"]
clean = clean[[x in keep_phases for x in clean.phasename]]

# Create columns enumerating overall rep number
subjects = clean.subject_id.unique()
n_subjects = len(clean.subject_id.unique())
n_overall_reps = list(range(1, int(clean.shape[0]/n_subjects)+1))*n_subjects
clean["n_overall_reps"] = n_overall_reps

# Pivot data
perturbation_f1 = clean.pivot(index="n_overall_reps", columns="subject_id", values="perturbation_value")
f1 = clean.pivot(index="n_overall_reps", columns="subject_id", values="Avg_F1")

# Identify males and females
max_perturbation = perturbation_f1.max(axis=0)
gender = ["F" if x > 0.30 else "M" for x in max_perturbation]
female_subjects = subjects[[i for i, x in enumerate(max_perturbation) if x > 0.3]]
male_subjects = subjects[[i for i, x in enumerate(max_perturbation) if x < 0.3]]
female_f1 = f1[female_subjects]
male_f1 = f1[female_subjects]

# Convert NaNs perturbation values to 0s
female_perturbation_f1 = perturbation_f1[2].fillna(0).tolist()
male_perturbation_f1 = perturbation_f1[3].fillna(0).tolist()

# Add perturbation values
female_f1.insert(loc=0, column='perturbation_value', value=female_perturbation_f1)
male_f1.insert(loc=0, column='perturbation_value', value=male_perturbation_f1)

In [9]:
# Export to .csv
female_f1.to_csv('data/female_IH.csv', index=False)
male_f1.to_csv('data/male_IH.csv', index=False)

In [130]:
# # Returned sorted file paths
# def getFilePaths(path):
#     files = glob.glob(path)
#     files.sort(key = sortKey)
#     return files

# def sortKey(path):
#     parts = path.split('_')
#     utterance = int(parts[7][1:])
#     return utterance

# # Get f0min and f0max
# def getPitchRange(files):
#     pitches = []
#     for file in files:
#         try:
#             sound = parselmouth.Sound(file)
#         except praat.ValueError:
#             print("Skipping: " + file + ", not a viable sound file")
#             continue
#         pitch = call(sound, "To Pitch", 0.0, 80, 450) #create a praat pitch object
#         meanF0 = call(pitch, "Get mean", 0, 0, "Hertz") # get mean pitch for each utterance
#         pitches.append(meanF0)
#     f0min = min(pitches)
#     f0max = max(pitches)
#     return f0min, f0max 

# # Get formants (from PraatScripts by https://github.com/drfeinberg/PraatScripts/, modified to remove median)
# def measureFormants(file, f0min, f0max):
#     sound = parselmouth.Sound(file) # read the sound
#     pitch = call(sound, "To Pitch (cc)", 0, f0min, 15, 'no', 0.03, 0.45, 0.01, 0.35, 0.14, f0max)
#     pointProcess = call(sound, "To PointProcess (periodic, cc)", f0min, f0max)
    
#     formants = call(sound, "To Formant (burg)", 0.0025, 5, 5000, 0.025, 50)
#     numPoints = call(pointProcess, "Get number of points")

#     f1_list = []
#     f2_list = []
#     f3_list = []
#     f4_list = []
    
#     # Measure formants only at glottal pulses
#     for point in range(0, numPoints):
#         point += 1
#         t = call(pointProcess, "Get time from index", point)
#         f1 = call(formants, "Get value at time", 1, t, 'Hertz', 'Linear')
#         f2 = call(formants, "Get value at time", 2, t, 'Hertz', 'Linear')
#         f3 = call(formants, "Get value at time", 3, t, 'Hertz', 'Linear')
#         f4 = call(formants, "Get value at time", 4, t, 'Hertz', 'Linear')
#         f1_list.append(f1)
#         f2_list.append(f2)
#         f3_list.append(f3)
#         f4_list.append(f4)
    
#     f1_list = [f1 for f1 in f1_list if str(f1) != 'nan']
#     f2_list = [f2 for f2 in f2_list if str(f2) != 'nan']
#     f3_list = [f3 for f3 in f3_list if str(f3) != 'nan']
#     f4_list = [f4 for f4 in f4_list if str(f4) != 'nan']
    
#     # calculate mean formants across pulses
#     try:
#         f1_mean = statistics.mean(f1_list)
#         f2_mean = statistics.mean(f2_list)
#         f3_mean = statistics.mean(f3_list)
#         f4_mean = statistics.mean(f4_list)
#     except statistics.StatisticsError:
#         print("Skipping: " + file + ", no formants extracted")
#         return None
    
#     return f1_mean, f2_mean, f3_mean, f4_mean

# # (F1_actual + F1_feedback)/F1_actual
# def getPerturbationValues(actual, feedback, f0min, f0max):
#     perturbation_values: Dict[str, float] = {
#         "f1": [],
#         "f2": [],
#         "f3": [],
#         "f4": [],
#     }
    
#     first_trial = True
    
#     for i in range(1, len(actual)):
#         print("Fetching formants from " + actual[i])
        
#         # Get formants
#         actual_formants = measureFormants(actual[i], f0min, f0max)
#         feedback_formants = measureFormants(feedback[i], f0min, f0max)
        
#         if actual_formants is None or feedback_formants is None:
#             continue
#         else:
#             (f1_actual, f2_actual, f3_actual, f4_actual) = actual_formants
#             (f1_feedback, f2_feedback, f3_feedback, f4_feedback) = feedback_formants
#             f1_perturb = (f1_actual + f1_feedback)/f1_actual
#             f2_perturb = (f2_actual + f2_feedback)/f2_actual
#             f3_perturb = (f3_actual + f3_feedback)/f3_actual
#             f4_perturb = (f4_actual + f4_feedback)/f4_actual
        
#         perturbation_values["f1"].append(f1_perturb)
#         perturbation_values["f2"].append(f2_perturb)
#         perturbation_values["f3"].append(f3_perturb)
#         perturbation_values["f4"].append(f4_perturb)
        
#     perturbation_values = pd.DataFrame(perturbation_values)
#     return perturbation_values

In [64]:
# # Get f0min and f0max of subject
# pre_files = glob.glob("data/pre/*_O_*.wav")
# (f0min, f0max) = getPitchRange(pre_files)

# # Get perturbation values
# actual = getFilePaths("data/ramp_up/*_O_*.wav")
# feedback = getFilePaths("data/ramp_up/*_I_*.wav")
# perturbation_values = getPerturbationValues(actual, feedback, f0min, f0max)

# # Get actual uttered formants

Fetching formants from data/ramp_up/S19_FIH_P5_R2_T1_F2444_U28_HECK_O_BSUBJECT19_20170130_230_[3].wav
Fetching formants from data/ramp_up/S19_FIH_P5_R3_T1_F2445_U29_HECK_O_BSUBJECT19_20170130_230_[3].wav
Fetching formants from data/ramp_up/S19_FIH_P5_R4_T1_F2446_U30_HECK_O_BSUBJECT19_20170130_230_[3].wav
Fetching formants from data/ramp_up/S19_FIH_P5_R5_T1_F2447_U31_HECK_O_BSUBJECT19_20170130_230_[3].wav
Fetching formants from data/ramp_up/S19_FIH_P5_R6_T1_F2448_U32_HECK_O_BSUBJECT19_20170130_230_[3].wav
Fetching formants from data/ramp_up/S19_FIH_P5_R7_T1_F2449_U33_HECK_O_BSUBJECT19_20170130_230_[3].wav
Fetching formants from data/ramp_up/S19_FIH_P5_R8_T1_F2450_U34_HECK_O_BSUBJECT19_20170130_230_[3].wav
Fetching formants from data/ramp_up/S19_FIH_P5_R9_T1_F2451_U35_HECK_O_BSUBJECT19_20170130_230_[3].wav
Fetching formants from data/ramp_up/S19_FIH_P5_R10_T1_F2452_U36_HECK_O_BSUBJECT19_20170130_230_[3].wav
Fetching formants from data/ramp_up/S19_FIH_P5_R11_T1_F2453_U37_HECK_O_BSUBJECT19

In [65]:
# perturbation_values

,f1,f2,f3,f4
0,2.010518,1.997708,1.999929,2.002247
1,2.019467,1.993306,2.002297,1.996530
2,2.036318,1.993548,1.999809,2.000987
3,2.043400,1.995849,2.001129,2.000109
4,2.027144,1.990193,2.000722,1.995142
5,2.037736,1.984877,1.998059,1.998907
6,2.053560,1.989700,1.999238,1.992994
7,2.073687,1.987811,1.998540,1.998421
8,2.049929,1.985560,1.999823,2.001831
9,2.131311,1.987523,2.001554,2.000625
